<a href="https://colab.research.google.com/github/Udit111111/ML---2/blob/main/Bayesian_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. INSTALLATION (Run this first to resolve the ModuleNotFoundError)
# The 'pgmpy' library is required for Bayesian Networks.
!pip install pgmpy

# --- Begin Bayesian Network Code ---

import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import pandas as pd

print("Libraries imported successfully.")
print("-" * 30)

# 2. Define the structure of the Bayesian Network
# Example: Cloudy -> Sprinkler, Cloudy -> Rain, Sprinkler -> WetGrass, Rain -> WetGrass
model = BayesianNetwork([
    ('Cloudy', 'Sprinkler'),
    ('Cloudy', 'Rain'),
    ('Sprinkler', 'WetGrass'),
    ('Rain', 'WetGrass')
])
#

# 3. Define the Conditional Probability Distributions (CPDs)
# States: 0=False, 1=True

# CPD for Cloudy (Root Node) - P(Cloudy)
# P(C=F), P(C=T)
cpd_cloudy = TabularCPD(
    variable='Cloudy',
    variable_card=2,
    values=[[0.5], [0.5]]
)

# CPD for Sprinkler - P(Sprinkler | Cloudy)
# States: Cloudy=0 (F), Cloudy=1 (T)
# [[P(S=F|C=F), P(S=F|C=T)],
#  [P(S=T|C=F), P(S=T|C=T)]]
cpd_sprinkler = TabularCPD(
    variable='Sprinkler',
    variable_card=2,
    values=[[0.5, 0.9],
            [0.5, 0.1]],
    evidence=['Cloudy'],
    evidence_card=[2]
)

# CPD for Rain - P(Rain | Cloudy)
# States: Cloudy=0 (F), Cloudy=1 (T)
cpd_rain = TabularCPD(
    variable='Rain',
    variable_card=2,
    values=[[0.8, 0.2],
            [0.2, 0.8]],
    evidence=['Cloudy'],
    evidence_card=[2]
)

# CPD for WetGrass - P(WetGrass | Sprinkler, Rain)
# Evidence Order: (S, R) = (F,F), (F,T), (T,F), (T,T)
# P(W=F | S, R) (Row 0), P(W=T | S, R) (Row 1)
cpd_wet_grass = TabularCPD(
    variable='WetGrass',
    variable_card=2,
    values=[[1.0, 0.1, 0.1, 0.01],
            [0.0, 0.9, 0.9, 0.99]],
    evidence=['Sprinkler', 'Rain'],
    evidence_card=[2, 2]
)

# Attach the CPDs to the model
model.add_cpds(cpd_cloudy, cpd_sprinkler, cpd_rain, cpd_wet_grass)

# Check if the model is valid
try:
    model.check_model()
    print("Bayesian Network structure and CPDs are valid.")
except:
    print("Error: Model validation failed.")
print("-" * 30)

# 4. Perform Inference using Variable Elimination
inference = VariableElimination(model)

# --- SCENARIO 1: Prediction with Simple Evidence ---
## Predict P(Rain | WetGrass=True)
print("Scenario 1: P(Rain | WetGrass=True)")
query_1 = inference.query(
    variables=['Rain'],
    evidence={'WetGrass': 1} # 1 = True
)
# Format and display the result nicely
prob_rain_given_wetgrass = pd.DataFrame(
    {'Rain State': ['False', 'True'],
     'Probability': query_1.values.flatten()},
    index=[0,1]
)
print(prob_rain_given_wetgrass)
print("\n")

# --- SCENARIO 2: Prediction with Multiple Evidence ---
## Predict P(Sprinkler | WetGrass=True, Cloudy=False)
print("Scenario 2: P(Sprinkler | WetGrass=True, Cloudy=False)")
query_2 = inference.query(
    variables=['Sprinkler'],
    evidence={'WetGrass': 1, 'Cloudy': 0} # 1 = True, 0 = False
)
# Format and display the result nicely
prob_sprinkler_given_evidence = pd.DataFrame(
    {'Sprinkler State': ['False', 'True'],
     'Probability': query_2.values.flatten()},
    index=[0,1]
)
print(prob_sprinkler_given_evidence)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 41.7 MB/s eta 0:00:00
